In [1]:
import pandas as pd
import os
import shutil
from Bio import SeqIO
from Bio import AlignIO
import distance
import editdistance
import math
import subprocess
import numpy as np

In [2]:
BASE_OUT_PATH = '/home/gamran/genome_analysis/Warrior/Richard/output/post_analysis/'
ALLELE_PATH = '/home/gamran/genome_analysis/Warrior/Richard/output/defining_alleles_2/allele_analysis/alleles_proteinortho_graph516/'
GENOME_PATH = '/home/gamran/genome_analysis/Warrior/Richard/output/genome_v03/'

PAML_PATH = os.path.join(BASE_OUT_PATH, 'paml')
if not os.path.exists(PAML_PATH):
    os.mkdir(PAML_PATH)

In [3]:
# Create DataFrame with all alleles
overlapDf = pd.read_csv(os.path.join(ALLELE_PATH, 'DK_0911_v03_p_ctg.h_contig_overlap.alleles'), header=None, names=['alleleOne', 'alleleTwo'], sep='\t')
noOverlapDf = pd.read_csv(os.path.join(ALLELE_PATH, 'DK_0911_v03_p_ctg.no_specific_h_contig_overlap.alleles'), header=None, names=['alleleOne', 'alleleTwo'], sep='\t')
diffContigDf = pd.read_csv(os.path.join(ALLELE_PATH, 'DK_0911_v03_p_ctg.no_respective_h_contig_overlap.alleles'), header=None, names=['alleleOne', 'alleleTwo'], sep='\t')

overlapDf['matchType'] = 'overlap'
noOverlapDf['matchType'] = 'no_overlap'
diffContigDf['matchType'] = 'different_pcontig'

alleleDf = overlapDf.append([noOverlapDf, diffContigDf])

alleleDf['folder'] = alleleDf.alleleOne + '_' + alleleDf.alleleTwo
alleleDf.set_index('folder', inplace=True)

assert(len(alleleDf) == len(overlapDf) + len(noOverlapDf) + len(diffContigDf))

In [4]:
# Combine haplotig & primary contig fasta files for each allele type
# (protein, gene, CDS), and change headings so that they use id 
# (e.g. evm.model.pcontig_057.39) instead of locus tag (e.g. DK0911_16805).

os.chdir('/home/gamran/genome_analysis/Warrior/Richard/scripts')
%run DK_0911_v03_dictionaries.ipynb
locusToIdDict = getLocusToIdDict()

def combineCtgFastaFiles(fastaFiles, fOut):
    '''combines fasta files and rewrites headings from locus tags
    (DK0911_16805) to id tags (evm.model.pcontig_057.39)'''
    with open(fOut, 'w') as outFile:
        for fastaFile in fastaFiles:
            with open(fastaFile, 'r') as inFile:
                for line in inFile:
                    if '>' in line:
                        line = '>' + locusToIdDict[line[1:-1]] + '\n'
                    outFile.write(line)
    return True

hProteinFasta = os.path.join(GENOME_PATH, 'DK_0911_v03_h_ctg.protein.fa')
pProteinFasta = os.path.join(GENOME_PATH, 'DK_0911_v03_p_ctg.protein.fa')
hGeneFasta = os.path.join(GENOME_PATH, 'DK_0911_v03_h_ctg.gene.fa')
pGeneFasta = os.path.join(GENOME_PATH, 'DK_0911_v03_p_ctg.gene.fa')
hCDSFasta = os.path.join(GENOME_PATH, 'DK_0911_v03_h_ctg.CDS.fa')
pCDSFasta = os.path.join(GENOME_PATH, 'DK_0911_v03_p_ctg.CDS.fa')

PH_PROTEIN_FASTA = os.path.join(GENOME_PATH, 'DK_0911_v03_ph_ctg.protein.fa')
PH_GENE_FASTA = os.path.join(GENOME_PATH, 'DK_0911_v03_ph_ctg.gene.fa')
PH_CDS_FASTA = os.path.join(GENOME_PATH, 'DK_0911_v03_ph_ctg.CDS.fa')

combineCtgFastaFiles([hProteinFasta, pProteinFasta], PH_PROTEIN_FASTA)
combineCtgFastaFiles([hGeneFasta, pGeneFasta], PH_GENE_FASTA)
combineCtgFastaFiles([hCDSFasta, pCDSFasta], PH_CDS_FASTA)

True

In [5]:
def getFastaDict(fastaFile):
    d = {}
    for gene in SeqIO.parse(fastaFile, 'fasta'):
        d[gene.id] = gene
    return d

SEQRECORD_PROTEIN_DICT = getFastaDict(PH_PROTEIN_FASTA)
SEQRECORD_GENE_DICT = getFastaDict(PH_GENE_FASTA)
SEQRECORD_CDS_DICT = getFastaDict(PH_CDS_FASTA)

In [6]:
# def getSpecificFasta(geneList, fastaFile):
#     '''Returns the fasta of gene in list as SeqIO object '''
#     # Ben's original code had CDS with '.model.' and 
#     # protein as '.TU.'
#     l = []
#     for gene in SeqIO.parse(fastaFile, 'fasta'):
#         if gene.id in geneList:
#             l.append(gene)
#     return l
# 
# def writeAllelicFasta(alleleOne, alleleTwo, alleleType, outPath):
#     '''writes fasta file containing fasta information for two alleles
#     in the outPath'''
#     assert(alleleType.upper() in ['CDS', 'GENE', 'PROTEIN'])
#     fastaFilePath = globals()['PH_' + alleleType.upper() + '_FASTA'] # e.g. alleleType = 'GENE', then fastaFile = PH_GENE_FASTA
#     specificFastaList = getSpecificFasta([alleleOne, alleleTwo], fastaFilePath)
#     with open(os.path.join(outPath, alleleType.lower() + '.fa'), 'w') as outFile:
#         SeqIO.write(specificFastaList, outFile, 'fasta')
#     return True

def writeAllelicFasta(alleleOne, alleleTwo, alleleType, outPath):
    '''writes fasta file containing fasta information for two alleles
    in the outPath'''
    assert(alleleType.upper() in ['CDS', 'GENE', 'PROTEIN'])
    
    seqRecordDict = globals()['SEQRECORD_' + alleleType.upper() + '_DICT']
    
    alleleSeqRecords = [seqRecordDict[alleleOne], seqRecordDict[alleleTwo]]
    with open(os.path.join(outPath, alleleType.lower() + '.fa'), 'w') as outFile:
        SeqIO.write(alleleSeqRecords, outFile, 'fasta')

    return True

def writeAlignmentScript(alleleOutPath, scriptLoc = os.path.join(PAML_PATH, 'paml_script.sh')):
    with open(scriptLoc, 'a') as outFile:
        print('cd %s' % alleleOutPath, file=outFile)
        print('/home/gamran/anaconda3/muscle3.8.31_i86linux64 -clwstrict -in protein.fa -out protein.aln', file=outFile)
        print('perl /home/gamran/anaconda3/pal2nal.v14/pal2nal.pl -output paml protein.aln cds.fa > cds_codon.aln', file=outFile)
        print('perl /home/gamran/anaconda3/pal2nal.v14/pal2nal.pl protein.aln cds.fa > cds_codon.clustal', file=outFile)
        print('cp %s/yn00.ctl ./' % PAML_PATH, file=outFile)
        print('/home/gamran/anaconda3/paml4.9g/bin/yn00', file=outFile)
    return True

In [7]:
def prepareAlignmentBashScript(scriptLoc = os.path.join(PAML_PATH, 'paml_script.sh')):
    with open(scriptLoc, 'w') as pamlScript:
        print('#!/bin/bash', file=pamlScript)

    for index, [alleleOne, alleleTwo, alleleType] in alleleDf.iterrows():

        alleleOutPath = os.path.join(PAML_PATH, '%s_%s' % (alleleOne, alleleTwo))
        if not os.path.exists(alleleOutPath):
            os.mkdir(os.path.join(PAML_PATH, '%s_%s' % (alleleOne, alleleTwo)))

        writeAllelicFasta(alleleOne, alleleTwo, 'CDS', alleleOutPath)
        writeAllelicFasta(alleleOne, alleleTwo, 'PROTEIN', alleleOutPath)

        writeAlignmentScript(alleleOutPath, os.path.join(PAML_PATH, 'paml_script.sh'))

In [8]:
# alleleDf['folder'] = alleleDf.alleleOne + '_' + alleleDf.alleleTwo
# alleleDf.index = alleleDf['folder']

In [9]:
def assignDistancesToAlleles(df, folder, alignmentFile, alleleType):
    '''Adds hamming and levenshtein distance columns to an allele pair
    (indexed by 'folder' name) in df'''
    assert(alleleType.upper() in ['PROTEIN', 'CDS', 'GENE'])
    seq1, seq2 = AlignIO.read(open(alignmentFile, 'r'), format='clustal', seq_count=2)
    seq1 = str(seq1.seq).upper()
    seq2 = str(seq2.seq).upper()
    assert(len(seq1) == len(seq2))
    df.loc[folder, alleleType.lower() + '_hamming'] = distance.hamming(seq1, seq2, normalized=True)
    df.loc[folder, alleleType.lower() + '_levenshtein'] = editdistance.eval(seq1, seq2)/len(seq1)
    return df

def assignDistancesToAllAlleles(alleleDf):
    count = 0
    total = len(alleleDf)
    percentDone = 0
    
    print("Calculating distances and adding them to the allele DataFrame...")
    
    for folder in alleleDf.index:

        proteinAlignmentFile = os.path.join(PAML_PATH, folder, 'protein.aln')
        alleleDf = assignDistancesToAlleles(alleleDf, folder, proteinAlignmentFile, 'PROTEIN')

        cdsAlignmentFile = os.path.join(PAML_PATH, folder, 'cds_codon.clustal')
        alleleDf = assignDistancesToAlleles(alleleDf, folder, cdsAlignmentFile, 'CDS')

        count += 1
        if round(count/total * 100) > percentDone:
            percentDone = round(count/total * 100)
            print("%s%% complete" % percentDone)
    return alleleDf

In [10]:
def parse_dNdS_to_df(line, alleleDf, folder, dNdS_label):
    dN = re.findall(r'dN = [-| ]?(.*) w', line)[0]
    dS = re.findall(r'dS = [-| ]?(.*) dN', line)[0]
    return assign_dNdS(dN, dS, alleleDf, folder, dNdS_label)

def assign_dNdS(dN, dS, alleleDf, folder, dNdS_label):
    if float(dS) > 0:
        alleleDf.loc[folder, dNdS_label] = float(dN)/float(dS)
    else:
        alleleDf.loc[folder, dNdS_label] = np.nan
    return alleleDf

def assign_dNdS_to_all_alleles(alleleDf):
    for folder in alleleDf.index:
        alleleYn = os.path.join(PAML_PATH, folder,'yn.out')
        with open(alleleYn, 'r') as ynOut:
            #now loop over the lines and parse out stuff
            for i, line in enumerate(ynOut):
                if line.startswith('seq. seq. ') and i > 0:
                    next(ynOut) # we want the line that is two after the line starting with 'seq. seq '
                    dataLine = next(ynOut)
                    dN = dataLine.split('+-')[0].rstrip().split(' ')[-1]
                    dS = dataLine.split('+-')[1].rstrip().split(' ')[-1]
                    alleleDf = assign_dNdS(dN, dS, alleleDf, folder, 'yn00_dN/dS')
                elif line.startswith('LWL85:') and 'nan' not in line:
                    alleleDf = parse_dNdS_to_df(line, alleleDf, folder, 'LWL85_dN/dS')
                elif line.startswith('LWL85m:') and 'nan' not in line:
                    alleleDf = parse_dNdS_to_df(line, alleleDf, folder, 'LWL85m_dN/dS')
                elif line.startswith('LPB93:') and 'nan' not in line:
                    alleleDf = parse_dNdS_to_df(line, alleleDf, folder, 'LPB93_dN/dS')
                else:
                    continue
    return alleleDf

In [ ]:
def loadDfIfExists():

In [11]:
def main():
    prepareAlignmentBashScript(os.path.join(PAML_PATH, 'paml_script.sh'))
    
    # !bash {os.path.join(PAML_PATH, 'paml_script.sh')}
    
    distDfPath = os.path.join(BASE_OUT_PATH, 'distanceDf.df')
    if os.path.exists(distDfPath) and os.path.getsize(distDfPath) > 0:
        print("DataFrame with distance calculations at %s appears to have already been generated. Reading in this dataframe instead of re-generating it." % distDfPath)
        alleleDf = pd.read_csv(distDfPath, sep='\t', index_col=0)
    else:
        alleleDf = assignDistancesToAllAlleles(alleleDf)
        alleleDf.to_csv(distDfPath, sep='\t')
        pd.util.testing.assert_frame_equal(alleleDf, pd.read_csv(distDfPath, sep='\t', index_col=0))
    
    alleleDf = assign_dNdS_to_all_alleles(alleleDf)
    
    return alleleDf

In [12]:
if __name__ == "__main__":
    alleleDf = main()

DataFrame with distance calculations at /home/gamran/genome_analysis/Warrior/Richard/output/post_analysis/distanceDf.df appears to have already been generated. Reading in this dataframe instead of re-generating it.


In [14]:
alleleDf

,alleleOne,alleleTwo,matchType,protein_hamming,protein_levenshtein,cds_hamming,cds_levenshtein,yn00_dN/dS,LWL85_dN/dS,LWL85m_dN/dS,LPB93_dN/dS
folder,,,,,,,,,,,
evm.model.pcontig_000.1_evm.model.hcontig_000_024.1,evm.model.pcontig_000.1,evm.model.hcontig_000_024.1,overlap,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN
evm.model.pcontig_000.100_evm.model.hcontig_000_058.5,evm.model.pcontig_000.100,evm.model.hcontig_000_058.5,overlap,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN
evm.model.pcontig_000.1000_evm.model.hcontig_000_078.13,evm.model.pcontig_000.1000,evm.model.hcontig_000_078.13,overlap,0.000000,0.000000,0.003333,0.003333,-0.000000,0.000000,0.000000,0.000000
evm.model.pcontig_000.1001_evm.model.hcontig_000_078.18,evm.model.pcontig_000.1001,evm.model.hcontig_000_078.18,overlap,0.913043,0.903890,0.848970,0.811594,0.556251,NaN,NaN,NaN
evm.model.pcontig_000.1002_evm.model.hcontig_000_078.16,evm.model.pcontig_000.1002,evm.model.hcontig_000_078.16,overlap,0.918367,0.916100,0.876795,0.848828,0.258478,NaN,NaN,NaN
evm.model.pcontig_000.1003_evm.model.hcontig_000_078.16,evm.model.pcontig_000.1003,evm.model.hcontig_000_078.16,overlap,0.887892,0.885650,0.845291,0.816891,0.212720,NaN,NaN,NaN
evm.model.pcontig_000.1005_evm.model.hcontig_000_078.13,evm.model.pcontig_000.1005,evm.model.hcontig_000_078.13,overlap,0.709677,0.701613,0.634409,0.602151,0.483521,0.467381,0.314064,0.456737
evm.model.pcontig_000.1007_evm.model.hcontig_000_078.16,evm.model.pcontig_000.1007,evm.model.hcontig_000_078.16,overlap,0.111111,0.111111,0.109599,0.108088,0.586777,0.514706,0.733333,0.732673
evm.model.pcontig_000.1008_evm.model.hcontig_000_060.2,evm.model.pcontig_000.1008,evm.model.hcontig_000_060.2,overlap,0.000000,0.000000,0.002976,0.002976,-0.000000,0.000000,0.000000,0.000000
